In [16]:
# !python3 -m pip install graphviz
import json
import pandas as pd
from graphviz import Digraph
from pathlib import Path

In [17]:
df = pd.read_csv("resources/2021-04-16/ncbi_sgene_good_unique.csv")
df

,sgene_nucleotide,accessions,accessions_count,accession
0,AGTCTCTAGTCAGTGTGTTAATCTTACAACCAGAACTCAATTACCC...,MT970416.1,1,1_07011ff250ef4a4ace2ef9b43d5c2ba1
1,ATGCTTGTTTTTTTTTTTGTTTTATTGCCACTAGTCTCTAGTCAGT...,MW540269.1|MW540268.1,2,2_059f461871887e99ab81c93d6be95c63
2,ATGTATGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...,MW467454.1,1,1_048ae97f946c3ec03e7a1e6e85022dd5
3,ATGTTAGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...,MT496989.1|MT535500.1,2,2_db94be2d798aced9967b7540b04c6e4f
4,ATGTTAGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...,MW206350.1|MW206333.1|MW420547.1|MW420251.1,4,4_d7f6e71507ff532913077e139a1200ee
...,...,...,...,...
4379,ATGTTTGTTTTTTTTTTTGTTTTATTGCCACTAGTCTCTAGTCAGT...,MW460643.1,1,1_1c35868a205decc006af1af9332de520
4380,ATGTTTTTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...,MT873167.1,1,1_f99d371f32ad020303cb3ca0946b9773
4381,ATGTTTTTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...,MT972160.1,1,1_85a663a10c0508eb03015b0d62f4a05b
4382,CAATTACCCCCTGCATACACTAATTCTTTCACACGTGGTGTTTATT...,MT970387.1,1,1_bc8364434f1c405884ffb19232630d57


In [18]:
def make_tree(working_dir, max_pairs=150):
    path = Path(working_dir)
    
    pairs = pd.read_csv(path.joinpath("pairs.csv"))
    dot = Digraph(comment=working_dir)
    for index, pair in pairs.iterrows():
        if index == max_pairs:
            break

        parent = df[df["accession"] == pair["parent"]]["accessions"].values[0].split("|")[0]
        child = df[df["accession"] == pair["child"]]["accessions"].values[0].split("|")[0]

        dot.edge(parent, child)

    dot.render(f"test-output/{path.stem}", format="png")
    dot.render(f"test-output/{path.stem}", format="pdf")

In [19]:
# make_tree("resources/2021-04-16/RAxML_aligned/")
# make_tree("resources/2021-04-16/levenshtein_aligned/")
# make_tree("resources/2021-04-16/levenshtein_not_aligned/")

In [30]:
def make_color_tree(working_dir1, working_dir2, max_pairs=150):
    path1 = Path(working_dir1)
    path2 = Path(working_dir2)
    
    pairs1 = pd.read_csv(path1.joinpath("pairs.csv"))
    pairs2 = pd.read_csv(path2.joinpath("pairs.csv"))
    
    merged = pd.merge(pairs1, pairs2, on=['parent', 'child'], how='left', indicator="exists")
    
    dot = Digraph(comment=working_dir1)
    for index, pair in merged.iterrows():
        if index == max_pairs:
            break

        parent = df[df["accession"] == pair["parent"]]["accessions"].values[0].split("|")[0]
        child = df[df["accession"] == pair["child"]]["accessions"].values[0].split("|")[0]
        
        if pair.exists == 'both':
            dot.node(parent, color="green")
            dot.node(child, color="green")
        dot.edge(parent, child)

#     dot.render(f"test-output/{path.stem}", format="png")
    dot.render(f"test-output/{path1.stem}", format="pdf", view=True)
    dot.render(f"test-output/{path1.stem}", format="pdf")
    
working_dir1 = "resources/2021-04-16/RAxML_aligned/"
working_dir2 = "resources/2021-04-16/levenshtein_aligned/"
working_dir3 = "resources/2021-04-16/levenshtein_not_aligned/"
make_color_tree(working_dir1, working_dir2, 9999)